In [1]:
import pandas as pd
import cobra
from collections import OrderedDict
from copy import deepcopy

from custom_functions_scRBA import *

#### Load data

In [2]:
df_enz = pd.read_excel('../scRBA/curation/enz_stoich_curation_2021-05-21.xlsx')

In [3]:
model = cobra.io.load_json_model('../scRBA/input/model/y834_hvd_v4_rba.json')

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-26


In [4]:
df_rnas = pd.read_excel('../scRBA/input/scRNA/RNA_stoich.xlsx')
df_rnas.index = df_rnas.RNAid.to_list()

In [5]:
df_elo = pd.read_excel('../scRBA/input/translation_elongation_machinery.xlsx')
df_elo = df_elo[df_elo.paralog.isnull()]

In [6]:
df_pro = pd.read_excel('../scRBA/curation/prot_stoich_curation_2021-05-21.xlsx')
df_pro.index = df_pro.id.to_list()

In [7]:
df_aamap = pd.read_csv('../scRBA/input/amino_acid_map.txt', sep='\t')
df_aamap.index = df_aamap.aa_abbv.to_list()

In [8]:
with open('../scRBA/input/rxns_req.txt') as f:
    rxns_req = f.read().split('\n')[:-1]
with open('../scRBA/input/proteins_req.txt') as f:
    pros_req = f.read().split('\n')[:-1]
with open('../scRBA/input/cofactors_req.txt') as f:
    cofs_req = f.read().split('\n')[:-1]

In [9]:
df_aa_dummy = pd.read_excel('../scRBA/input/dummy_prot_calc.xlsx')
df_aa_dummy.index = df_aa_dummy.aa_abbv.to_list()

#### Assemble reactions
Remove interactive support subunit and reactions that synthesize them

In [10]:
df_eqn = pd.DataFrame(columns=['id', 'type', 'symbolic_coeffs', 'reaction'])
c = df_eqn.shape[0] - 1

### Metabolic network reaction
for i in df_enz.index:
    rxn_id = df_enz.id[i]
    if rxn_id in rxns_req:
        continue
        
    c += 1
    _,rxn_base_id,rxn_dir,enz_id = extract_details_from_rxnid(rxn_id)
    
    rxn_base = model.reactions.get_by_id(rxn_base_id)
    
    met_dict = metabolites_dict_from_reaction_equation_RBA(rxn_base.reaction)
    met_dict = {k:v for k,v in met_dict.items() if k != ''}
    met_dict = {'MET-' + k:v for k,v in met_dict.items()}
    if rxn_dir == 'REV':
        met_dict = {k:-v for k,v in met_dict.items()}
    elif rxn_dir == 'FWD':
        None
    else:
        print("Unknown ID that indicate reaction direction, only accepting 'FWD' and 'REV'")
    
    if enz_id != 'SPONT':
        met_dict['ENZ-' + enz_id] = '-mu/keff'
        df_eqn.loc[c, 'symbolic_coeffs'] = 'mu/keff'
    
    df_eqn.loc[c, 'id'] = rxn_id
    df_eqn.loc[c, 'type'] = 'metabolic'
    df_eqn.loc[c, 'reaction'] = build_reaction_equation_from_metabolites_dict_RBA(met_dict, arrow='-->')
    
### Enzyme synthesis network reaction
enz_stoich = OrderedDict()
for i in df_enz.index:
    if df_enz.enz[i] not in pros_req:
        enz_stoich[df_enz.enz[i]] = df_enz.protein_stoich[i]

c = df_eqn.shape[0] - 1
for enz_id,prot_str in enz_stoich.items():
    if prot_str == 'zeroCost':
        continue
    
    c += 1
    prot_str = prot_str.split(',')
    coeffs = OrderedDict({'PRO-' + i.split(':')[0]:-int(i.split(':')[1]) for i in prot_str})
    coeffs['ENZ-'+enz_id] = 1
    
    df_eqn.loc[c, 'id'] = 'ENZSYN-' + enz_id
    df_eqn.loc[c, 'type'] = 'enzyme'
    df_eqn.loc[c, 'reaction'] = build_reaction_equation_from_metabolites_dict_RBA(coeffs, arrow='-->')
    
"""# Dummy enzyme
c += 1
df_eqn.loc[c, 'id'] = 'ENZSYN-DUMMYENZ_c'
df_eqn.loc[c, 'type'] = 'enzyme'
df_eqn.loc[c, 'reaction'] = 'PRO-DUMMYPRO_c --> ENZ-DUMMYENZ_c'

# Dummy enzyme
c += 1
df_eqn.loc[c, 'id'] = 'ENZSYN-DUMMYENZ_m'
df_eqn.loc[c, 'type'] = 'enzyme'
df_eqn.loc[c, 'reaction'] = 'PRO-DUMMYPRO_m --> ENZ-DUMMYENZ_m'""";
    
### Ribosome
rnas = ['rrna18s_c', 'rrna5s_c', 'rrna58s_c', 'rrna25s_c']
rnas_mw = {'rrna18s_c': 608.60, 'rrna5s_c': 40.93, 'rrna58s_c': 53.26, 'rrna25s_c': 1151.85}
c = df_eqn.shape[0] - 1
for rna in rnas:
    c += 1
    rna_stoich = OrderedDict({i:0 for i in ['MET-'+rna, 'MET-atp_c', 'MET-ctp_c',
                                            'MET-gtp_c', 'MET-utp_c', 'MET-ppi_c']})
    rna_stoich['MET-'+rna] = 1
    rna_stoich['MET-atp_c'] = -int(df_rnas.A[rna])
    rna_stoich['MET-ctp_c'] = -int(df_rnas.C[rna])
    rna_stoich['MET-gtp_c'] = -int(df_rnas.G[rna])
    rna_stoich['MET-utp_c'] = -int(df_rnas.U[rna])
    rna_stoich['MET-ppi_c'] = int(df_rnas.loc[rna, ['A','C','G','U']].sum())
    rna_stoich['BIO-rna_c'] = rnas_mw[rna]
    
    df_eqn.loc[c, 'id'] = 'RIBO-' + rna
    df_eqn.loc[c, 'type'] = 'ribosome'
    df_eqn.loc[c, 'reaction'] = build_reaction_equation_from_metabolites_dict_RBA(rna_stoich, arrow='-->')
    
c += 1
ribo_stoich = OrderedDict()
ribo_stoich['MET-ribosome_c'] = 1
rnas = ['rrna18s_c', 'rrna5s_c', 'rrna58s_c', 'rrna25s_c']
for i in df_elo.index:
    if df_elo.id[i] in rnas:
        ribo_stoich['MET-' + df_elo.id[i]] = -1
    else:
        ribo_stoich['PRO-' + df_elo.id[i]] = -1
#ribo_stoich['BIO-rna_c'] = 1854.64
        
df_eqn.loc[c, 'id'] = 'RIBO-ribosome_c'
df_eqn.loc[c, 'type'] = 'ribosome'
df_eqn.loc[c, 'reaction'] = build_reaction_equation_from_metabolites_dict_RBA(ribo_stoich, arrow='-->')

# Active ribosome and ribosome biomass
c += 1
ribo_active = OrderedDict()
ribo_active['MET-ribosome_c'] = -1
ribo_active['MET-riboactive_c'] = 'friboactive'
df_eqn.loc[c, 'symbolic_coeffs'] = 'friboactive'

df_eqn.loc[c, 'id'] = 'RIBO-riboactive_c'
df_eqn.loc[c, 'type'] = 'ribosome'
df_eqn.loc[c, 'reaction'] = build_reaction_equation_from_metabolites_dict_RBA(ribo_active, arrow='-->')

### Protein
c = df_eqn.shape[0] - 1
for i in df_pro.index:
    if df_pro.id[i] in pros_req:
        continue
        
    c += 1
    prot_st = OrderedDict()
    for met in ['MET-fmettrna_c', 'MET-atp_c', 'MET-h2o_c', 'MET-mettrna_c',
                'MET-for_c', 'MET-adp_c', 'MET-pi_c', 'MET-h_c', 'MET-gtp_c',
                'MET-gdp_c']:
        prot_st[met] = 0

    seq = df_pro.sequence[i][:-1]
    for aa in df_aamap.index:
        prot_st[df_aamap.tRNA_in[aa]] = -seq.count(aa)
        prot_st[df_aamap.tRNA_out[aa]] = seq.count(aa)
        
    cofs_str = df_pro.cofactor_stoich[i]
    if pd.isnull(cofs_str) == False:
        cofs_st = cofs_str.split(',')
        cofs_st = OrderedDict({c.split(':')[0]:c.split(':')[1] for c in cofs_st})
        for k,v in cofs_st.items():
            if k not in cofs_req:
                prot_st['MET-' + k] = -int(v)
                
    prot_st['PRO-' + df_pro.id[i]] = 1
    prot_st['MET-riboactive_c'] = '-mu*NAA/kribo'
    
    if df_pro.subloc_assigned[i] in ['m', 'mm']:
        prot_st['PRO-preModeledProtBiom_m'] = df_pro.loc[i, 'MW (g/mmol)']
    else:
        prot_st['PRO-preModeledProtBiom_c'] = df_pro.loc[i, 'MW (g/mmol)']
    
    df_eqn.loc[c, 'symbolic_coeffs'] = 'mu*NAA/kribo'
    for met in ['MET-fmettrna_c', 'MET-atp_c', 'MET-h2o_c']:
        prot_st[met] -= 1
    for met in ['MET-mettrna_c', 'MET-for_c', 'MET-adp_c', 'MET-pi_c', 'MET-h_c']:
        prot_st[met] += 1
                
    for met in ['MET-gtp_c', 'MET-h2o_c']:
        prot_st[met] -= 2*len(seq)
    for met in ['MET-gdp_c', 'MET-pi_c', 'MET-h_c']:
        prot_st[met] += 2*len(seq)
    
    df_eqn.loc[c, 'id'] = 'PROSYN-' + df_pro.id[i]
    df_eqn.loc[c, 'type'] = 'protein'
    df_eqn.loc[c, 'reaction'] = build_reaction_equation_from_metabolites_dict_RBA(prot_st, arrow='-->')
    
# Dummy protein
dummyL = 405; dummyMW = 44.465469;
for dummy in ['DUMMYPRO_c', 'DUMMYPRO_m']:
    prot_st = OrderedDict()
    for met in ['MET-fmettrna_c', 'MET-atp_c', 'MET-h2o_c', 'MET-mettrna_c',
                'MET-for_c', 'MET-adp_c', 'MET-pi_c', 'MET-h_c', 'MET-gtp_c',
                'MET-gdp_c']:
        prot_st[met] = 0

    seq = df_pro.sequence[i][:-1]
    for aa in df_aamap.index:
        prot_st[df_aamap.tRNA_in[aa]] = -round(df_aa_dummy.N_AA[aa], 4)
        prot_st[df_aamap.tRNA_out[aa]] = round(df_aa_dummy.N_AA[aa], 4)

    prot_st['PRO-' + dummy] = 1
    prot_st['MET-riboactive_c'] = '-mu*NAA/kribo'
    
    for met in ['MET-fmettrna_c', 'MET-atp_c', 'MET-h2o_c']:
        prot_st[met] -= 1
    for met in ['MET-mettrna_c', 'MET-for_c', 'MET-adp_c', 'MET-pi_c', 'MET-h_c']:
        prot_st[met] += 1

    for met in ['MET-gtp_c', 'MET-h2o_c']:
        prot_st[met] -= 2*dummyL
    for met in ['MET-gdp_c', 'MET-pi_c', 'MET-h_c']:
        prot_st[met] += 2*dummyL
    
    c += 1
    prot_st['PRO-preUnmodeledProtBiom'] = dummyMW
    df_eqn.loc[c, 'symbolic_coeffs'] = 'mu*NAA/kribo'
    df_eqn.loc[c, 'id'] = 'PROSYN-' + dummy
    df_eqn.loc[c, 'type'] = 'protein'
    df_eqn.loc[c, 'reaction'] = build_reaction_equation_from_metabolites_dict_RBA(prot_st,
                                                            arrow='-->', floatdecimal=6)

### Dummy protein object sink
c += 1
df_eqn.loc[c, 'id'] = 'BIODIL-SINKDUMMY_c'
df_eqn.loc[c, 'type'] = 'biomass'
df_eqn.loc[c, 'reaction'] = 'PRO-DUMMYPRO_c --> '

c += 1
df_eqn.loc[c, 'id'] = 'BIODIL-SINKDUMMY_m'
df_eqn.loc[c, 'type'] = 'biomass'
df_eqn.loc[c, 'reaction'] = 'PRO-DUMMYPRO_m --> '

### Biomass
c += 1
df_eqn.loc[c, 'id'] = 'BIODIL-PRO1'
df_eqn.loc[c, 'type'] = 'biomass'
df_eqn.loc[c, 'reaction'] = 'PRO-preModeledProtBiom_m --> BIO-modeledProtBiom_m'

c += 1
df_eqn.loc[c, 'id'] = 'BIODIL-PRO2'
df_eqn.loc[c, 'type'] = 'biomass'
df_eqn.loc[c, 'reaction'] = 'PRO-preModeledProtBiom_c --> BIO-modeledProtBiom_m'

c += 1
df_eqn.loc[c, 'id'] = 'BIODIL-PRO3'
df_eqn.loc[c, 'type'] = 'biomass'
df_eqn.loc[c, 'reaction'] = 'PRO-preUnmodeledProtBiom --> BIO-modeledProtBiom_m'

c += 1
df_eqn.loc[c, 'id'] = 'BIODIL-PRO4'
df_eqn.loc[c, 'type'] = 'biomass'
df_eqn.loc[c, 'reaction'] = 'PRO-preModeledProtBiom_c --> BIO-modeledProtBiom_c'

c += 1
df_eqn.loc[c, 'id'] = 'BIODIL-PRO5'
df_eqn.loc[c, 'type'] = 'biomass'
df_eqn.loc[c, 'reaction'] = 'PRO-preUnmodeledProtBiom --> BIO-modeledProtBiom_c'

c += 1
df_eqn.loc[c, 'id'] = 'BIODIL-PRO6'
df_eqn.loc[c, 'type'] = 'biomass'
df_eqn.loc[c, 'reaction'] = 'PRO-preUnmodeledProtBiom --> BIO-unmodeledProtBiom'

c += 1
df_eqn.loc[c, 'id'] = 'BIODIL-RNA'
df_eqn.loc[c, 'type'] = 'biomass'
df_eqn.loc[c, 'reaction'] = '0.233979 MET-atp_c + 0.227364 MET-ctp_c + 0.233979 MET-gtp_c + 0.304678 MET-utp_c --> 0.319459 BIO-rna_c + MET-ppi_c'

c += 1
df_biom = pd.read_excel('../scRBA/input/biomass/biomass_RBA_2021-05-18_fermentation.xlsx')
df_biom = df_biom[df_biom.type.str.contains('intermediate') == False]
biom_st = OrderedDict()
symcoeffs = []
for i in df_biom.index:
    if df_biom.type[i] == 'constant':
        biom_st[df_biom.Metabolite[i]] = df_biom.linearConstant[i]
        
    elif df_biom.type[i] == 'symbolic':
        if df_biom.sign[i] == 'neg':
            biom_st[df_biom.Metabolite[i]] = '-' + df_biom.coeff_id[i]
            symcoeffs.append(df_biom.coeff_id[i])
        elif df_biom.sign[i] == 'pos':
            biom_st[df_biom.Metabolite[i]] = df_biom.coeff_id[i]
            symcoeffs.append(df_biom.coeff_id[i])
        else:
            print("Unrecoginzed sign, only recognize 'pos' or 'neg' string for sign assignment")
df_eqn.loc[c, 'id'] = 'BIODIL-BIODILFERMENTATION'
df_eqn.loc[c, 'type'] = 'biomass'
df_eqn.loc[c, 'symbolic_coeffs'] = ','.join(symcoeffs)
df_eqn.loc[c, 'reaction'] = build_reaction_equation_from_metabolites_dict_RBA(biom_st,
                                                        arrow='-->', floatdecimal=6)

c += 1
df_biom = pd.read_excel('../scRBA/input/biomass/biomass_RBA_2021-05-18_respiration.xlsx')
df_biom = df_biom[df_biom.type.str.contains('intermediate') == False]
biom_st = OrderedDict()
symcoeffs = []
for i in df_biom.index:
    if df_biom.type[i] == 'constant':
        biom_st[df_biom.Metabolite[i]] = df_biom.linearConstant[i]
        
    elif df_biom.type[i] == 'symbolic':
        if df_biom.sign[i] == 'neg':
            biom_st[df_biom.Metabolite[i]] = '-' + df_biom.coeff_id[i]
            symcoeffs.append(df_biom.coeff_id[i])
        elif df_biom.sign[i] == 'pos':
            biom_st[df_biom.Metabolite[i]] = df_biom.coeff_id[i]
            symcoeffs.append(df_biom.coeff_id[i])
        else:
            print("Unrecoginzed sign, only recognize 'pos' or 'neg' string for sign assignment")
df_eqn.loc[c, 'id'] = 'BIODIL-BIODILRESPIRATION'
df_eqn.loc[c, 'type'] = 'biomass'
df_eqn.loc[c, 'symbolic_coeffs'] = ','.join(symcoeffs)
df_eqn.loc[c, 'reaction'] = build_reaction_equation_from_metabolites_dict_RBA(biom_st,
                                                        arrow='-->', floatdecimal=6)

### For debugging purpose
c += 1
df_eqn.loc[c, 'id'] = 'DMRXNS-BIO-unmodeledProtBiom'
df_eqn.loc[c, 'type'] = 'debug'
df_eqn.loc[c, 'reaction'] = 'BIO-unmodeledProtBiom --> '

c += 1
df_eqn.loc[c, 'id'] = 'DMRXNS-BIO-modeledProtBiom_c'
df_eqn.loc[c, 'type'] = 'debug'
df_eqn.loc[c, 'reaction'] = 'BIO-modeledProtBiom_c --> '

c += 1
df_eqn.loc[c, 'id'] = 'DMRXNS-BIO-modeledProtBiom_m'
df_eqn.loc[c, 'type'] = 'debug'
df_eqn.loc[c, 'reaction'] = 'BIO-modeledProtBiom_m --> '

c += 1
df_eqn.loc[c, 'id'] = 'DMRXNS-BIO-rna_c'
df_eqn.loc[c, 'type'] = 'debug'
df_eqn.loc[c, 'reaction'] = 'BIO-rna_c --> '

In [11]:
df_eqn.to_excel('./scRAM_network_frompy.xlsx', index=None)

#### Assemble species list

In [12]:
met_list_raw = []
for i in df_eqn.index:
    met_dict = metabolites_dict_from_reaction_equation_RBA(df_eqn.reaction[i])
    met_list_raw += met_dict.keys()

met_list = sorted(list(set(met_list_raw)))
met_list = [i for i in met_list if i != '']

In [13]:
with open('./species.txt', 'w') as f:
    f.write('\n'.join(met_list))

#### Assemble coefficients

In [14]:
df_rxns = pd.read_excel('./scRAM_network_frompy.xlsx')
df_rxns.index = df_rxns.id.to_list()
df_rxns = df_rxns[df_rxns.symbolic_coeffs.isnull() == False]

df_keff = pd.read_excel('./keff_frompy.xlsx')
df_keff.index = df_keff.rxn_id.to_list()

df_pro = pd.read_excel('../scRBA/curation/prot_stoich_curation_2021-05-21.xlsx')
df_pro.index = df_pro.id.to_list()

In [15]:
df_params = pd.read_excel('../scRBA/input/params_respiration.xlsx')
df_params.index = df_params.param_id.to_list()

In [16]:
stoich = OrderedDict()

Constant part

In [17]:
for i in df_rxns.index:
    rxn_id = df_rxns.id[i]
    met_dict = metabolites_dict_from_reaction_equation_RBA(df_rxns.reaction[i])
    met_dict = {k:v for k,v in met_dict.items() if k != ''}
    for k,v in met_dict.items():
        if is_number(v):
            stoich["'" + k + "'.'" + rxn_id + "'"] = v

Symbolic part

In [18]:
kribo = 10.5*3600; dummyNAA = 405;

In [19]:
for i in df_rxns.index:
    met_dict = metabolites_dict_from_reaction_equation_RBA(df_rxns.reaction[i])
    met_dict = {k:v for k,v in met_dict.items() if k != '' and is_number(v) == False}
    sym = df_rxns.symbolic_coeffs[i]
    
    for k,v in met_dict.items():
        if v == '-mu/keff':
            v = str(round(-1 / df_keff.keff[i] / 3600, 12)) + '*mu'
        elif v == '-mu*NAA/kribo':
            pro_id = i[7:]
            if pro_id in ['DUMMYPRO_c', 'DUMMYPRO_m']:
                v = str(-round(dummyNAA / kribo, 12)) + '*mu'
            else:
                v = str(-round((len(df_pro.sequence[pro_id]) - 1) / kribo, 12)) + '*mu'
        else:
            v = df_params.string[v]
            
        stoich["'" + k + "'.'" + i + "'"] = str(v)

Fix stoichiometry of respiration and fermentation phenotype biomass dilution reaction

In [20]:
stoich["'BIO-unmodeledProtBiom'.'BIODIL-BIODILRESPIRATION'"] = '-(0.118970 + 0.139017*mu)'
stoich["'BIO-modeledProtBiom_c'.'BIODIL-BIODILRESPIRATION'"] = '-(0.121007 + 0.141397*mu)'
stoich["'BIO-modeledProtBiom_m'.'BIODIL-BIODILRESPIRATION'"] = '-(0.131677 + 0.153865*mu)'

stoich["'BIO-unmodeledProtBiom'.'BIODIL-BIODILFERMENTATION'"] = '-(0.118970 + 0.139017*mu)'
stoich["'BIO-modeledProtBiom_c'.'BIODIL-BIODILFERMENTATION'"] = '-(0.234957 + 0.274546*mu)'
stoich["'BIO-modeledProtBiom_m'.'BIODIL-BIODILFERMENTATION'"] = '-(0.017728 + 0.020715*mu)'

Save<br>
For mu < 0.211

In [21]:
import json

In [22]:
with open('./symbolic_stoich/sij_lower_0_211.json', 'w') as f: 
    json.dump(stoich, f)

For mu > 0.211

In [23]:
stoich_higher = deepcopy(stoich)
stoich_higher["'BIO-unmodeledProtBiom'.'BIODIL-BIODILRESPIRATION'"] = -0.148303012202
stoich_higher["'BIO-modeledProtBiom_c'.'BIODIL-BIODILRESPIRATION'"] = -0.150841828568
stoich_higher["'BIO-modeledProtBiom_m'.'BIODIL-BIODILRESPIRATION'"] = -0.164142817229

stoich_higher["'BIO-unmodeledProtBiom'.'BIODIL-BIODILFERMENTATION'"] = -0.148303012202
stoich_higher["'BIO-modeledProtBiom_c'.'BIODIL-BIODILFERMENTATION'"] = -0.292885824511
stoich_higher["'BIO-modeledProtBiom_m'.'BIODIL-BIODILFERMENTATION'"] = -0.022098821287

stoich_higher["'MET-mannan_c'.'BIODIL-BIODILRESPIRATION'"] = -0.539010001587
stoich_higher["'MET-13BDglucan_en'.'BIODIL-BIODILRESPIRATION'"] = -0.836427660720
stoich_higher["'MET-16BDglucan_en'.'BIODIL-BIODILRESPIRATION'"] = -0.220112542295
stoich_higher["'MET-chtn_c'.'BIODIL-BIODILRESPIRATION'"] = -0.019775196008
stoich_higher["'MET-glycogen_c'.'BIODIL-BIODILRESPIRATION'"] = -0.403253827394
stoich_higher["'MET-tre_c'.'BIODIL-BIODILRESPIRATION'"] = -0.020213206243
stoich_higher["'BIO-rna_c'.'BIODIL-BIODILRESPIRATION'"] = -0.07219476

stoich_higher["'MET-mannan_c'.'BIODIL-BIODILFERMENTATION'"] = -0.539010001587
stoich_higher["'MET-13BDglucan_en'.'BIODIL-BIODILFERMENTATION'"] = -0.836427660720
stoich_higher["'MET-16BDglucan_en'.'BIODIL-BIODILFERMENTATION'"] = -0.220112542295
stoich_higher["'MET-chtn_c'.'BIODIL-BIODILFERMENTATION'"] = -0.019775196008
stoich_higher["'MET-glycogen_c'.'BIODIL-BIODILFERMENTATION'"] = -0.403253827394
stoich_higher["'MET-tre_c'.'BIODIL-BIODILFERMENTATION'"] = -0.020213206243
stoich_higher["'BIO-rna_c'.'BIODIL-BIODILFERMENTATION'"] = -0.07219476

In [24]:
with open('./symbolic_stoich/sij_higher_0_211.json', 'w') as f: 
    json.dump(stoich_higher, f)